# Лабораторные работы №2-5
## Машинное обучение и анализ данных социальных сетей

# Работа №2: Кластеризация текстовых данных

In [ ]:
import json
import os
import numpy as np
import pandas as pd
import nltk
import re
import matplotlib.pyplot as plt

# Загрузка текстовых данных
text_messages_clean = []

with open(r"posts.json") as file:
    for line in file.readlines():
        wall = json.loads(line)
        parsed_response = json.JSONDecoder().decode(json.dumps(wall))
        nodes = []
        for key, value in parsed_response.items():
            nodes.append(value.get('items'))
        for node in nodes:
            for post in node:
                text = post.get("text")
                if text is not None:
                    text_messages_clean.append(text)

print(f"Загружено сообщений: {len(text_messages_clean)}")

In [ ]:
# Подсчет слов
words = {}
for message in text_messages_clean:
    if message is None:
        continue
    for word in message.split():
        if word not in words:
            words[word] = 1
        else:
            words[word] += 1

words = dict(sorted(words.items(), key=lambda item: item[1], reverse=True))
print(f"Уникальных слов: {len(words)}")
print("Топ-10 слов:", list(words.items())[:10])

In [ ]:
# TF-IDF и кластеризация
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, DBSCAN

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian")

def token_and_stem(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = [token for token in tokens if re.search('[а-яА-Я]', token)]
    return [stemmer.stem(t) for t in filtered_tokens]

stopwords = nltk.corpus.stopwords.words('russian')
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.75, max_features=10000, min_df=0.01,
    stop_words=stopwords, use_idf=True, tokenizer=token_and_stem, ngram_range=(1,3)
)

tfidf_matrix = tfidf_vectorizer.fit_transform(text_messages_clean)
print(f"TF-IDF матрица: {tfidf_matrix.shape}")

# Кластеризация
km = KMeans(n_clusters=5, random_state=42)
km.fit(tfidf_matrix)
print(f"K-means кластеры: {pd.Series(km.labels_).value_counts().to_dict()}")

# Работа №3: Кластеризация изображений

In [ ]:
# Класс для работы с изображениями
import cv2
from sklearn.cluster import KMeans as KMeansImage

class DominantColors:
    def __init__(self, image_path, clusters):
        self.CLUSTERS = clusters
        self.IMAGE_PATH = image_path
        self.IMAGE = None
        self.COLORS = None
        self.LABELS = None
    
    def dominantColors(self):
        try:
            img = cv2.imread(self.IMAGE_PATH)
            if img is None:
                raise ValueError(f"Не удалось загрузить: {self.IMAGE_PATH}")
            
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = img.reshape((img.shape[0] * img.shape[1], 3))
            self.IMAGE = img
            
            kmeans = KMeansImage(n_clusters=self.CLUSTERS, random_state=42, n_init=10)
            kmeans.fit(img)
            
            self.COLORS = kmeans.cluster_centers_
            self.LABELS = kmeans.labels_
            return self.COLORS.astype(int)
        except Exception as e:
            print(f"Ошибка: {e}")
            return None

print("Класс DominantColors создан")

# Работа №4: Визуализация графов

In [ ]:
# Импорт библиотек для графов
import vk_api
import time
import pickle
import networkx as nx

# Функция для получения друзей
def get_groups_user(friends_list, vk_tools):
    friends_list_out = {}
    for friend in friends_list:
        try:
            friends_list_out[friend] = vk_tools.get_all('friends.get', 100, {'user_id': friend})
        except Exception as e:
            friends_list_out[friend] = []
        time.sleep(1)
    return friends_list_out

print("Функции для работы с VK API загружены")
print("Примечание: Для использования требуется действительный токен")


In [ ]:
# Функция для создания графа
def make_graph(friends_out, friends_friends, user_id):
    graph = nx.Graph()
    graph.add_node(user_id, size=friends_out[user_id]['count'])
    
    for i in friends_out[user_id]['items']:
        try:
            if len(friends_friends[i]) == 0:
                continue
            graph.add_node(i, size=friends_friends[i]['count'])
            intersection = set(friends_out[user_id]['items']).intersection(set(friends_friends[i]['items']))
            graph.add_edge(user_id, i, weight=len(intersection))
        except:
            continue
    
    for i in range(len(friends_out[user_id]['items'])):
        id1 = friends_out[user_id]['items'][i]
        for k in range(i + 1, len(friends_out[user_id]['items'])):
            id2 = friends_out[user_id]['items'][k]
            try:
                if len(friends_friends[id1]) == 0 or len(friends_friends[id2]) == 0:
                    continue
                intersection = set(friends_friends[id1]['items']).intersection(set(friends_friends[id2]['items']))
                if len(intersection) > 0:
                    graph.add_edge(id1, id2, weight=len(intersection))
            except:
                continue
    
    return graph

print("Функция make_graph создана")


# Работа №5: Наивный байесовский классификатор

In [ ]:
# Наивный байесовский классификатор
# Требуется файл Mandrill.xlsx

# Этапы реализации:
# 1. Токенизация твитов (преобразование букв, удаление пунктуации)
# 2. Создание листов AppTokens и OtherTokens
# 3. Разделение твитов на токены с помощью формул НАЙТИ и ПСТР
# 4. Создание сводных таблиц AppTokensProbability и OtherTokensProbability
# 5. Расчет условных вероятностей каждого токена
# 6. Применение аддитивного сглаживания (сглаживание Лапласа)
# 7. Расчет натуральных логарифмов вероятностей
# 8. Тестирование на тестовом наборе твитов
# 9. Классификация твитов путем сравнения вероятностей

print("Наивный байесовский классификатор")
print("="*50)
print("Этапы реализации:")
print("1. Токенизация твитов")
print("2. Расчет условных вероятностей")
print("3. Применение сглаживания Лапласа")
print("4. Расчет логарифмов вероятностей")
print("5. Классификация новых твитов")
print("\nИспользуемые функции Excel/Calc:")
print("- СТРОЧН: преобразование букв в строчные")
print("- ПОДСТАВИТЬ: замена символов")
print("- НАЙТИ: поиск позиции пробела")
print("- ПСТР: извлечение подстроки (токена)")
print("- ДЛСТР: длина строки")
print("- ЕСЛИОШИБКА: обработка ошибок")
print("- ВПР: поиск значений в таблице")
print("- Сводные таблицы: подсчет частот токенов")


In [ ]:
# Пример формул для Excel/Calc
formulas = {
    "Преобразование в строчные": "=СТРОЧН(A2)",
    "Замена пунктуации": "=ПОДСТАВИТЬ(B2;\".\"; \" \")",
    "Поиск позиции пробела": "=ЕСЛИОШИБКА(НАЙТИ(\" \";A152;B2+1);ДЛСТР(A152)+1)",
    "Извлечение токена": "=ЕСЛИОШИБКА(ПСТР(A2;B2+1;B152-B2-1);\".\") ",
    "Длина токена": "=ДЛСТР(C2)",
    "Вероятность токена": "=C6/C$828",
    "Натуральный логарифм": "=LN(D6)",
    "Поиск вероятности по токену": "=ЕСЛИ(ДЛСТР(D2)<=3;0;ЕСЛИ(ЕНД(ВПР(D2;$AppTokensProbability.$A$5:$E$827;5;0));LN(1/$AppTokensProbability.$C$828);ВПР(D2;$AppTokensProbability.$A$5:$E$827;5;0)))"
}

print("Основные формулы для классификатора:")
print("="*70)
for name, formula in formulas.items():
    print(f"\n{name}:")
    print(f"  {formula}")


In [ ]:
# Эффективность классификатора
print("\nЭффективность наивного байесовского классификатора:")
print("="*70)
print()
print("Преимущества:")
print("  ✓ Простая реализация и интерпретация")
print("  ✓ Эффективен при работе с текстовыми данными")
print("  ✓ Хорошие результаты на практике")
print("  ✓ Масштабируемость")
print()
print("Ограничения:")
print("  ✗ Предположение об условной независимости признаков")
print("  ✗ Проблема с редкими словами (решается сглаживанием)")
print("  ✗ Неправильная оценка вероятностей в крайних случаях")
print()
print("Применение сглаживания Лапласа:")
print("  P(token|class) = (count + 1) / (total_tokens + |V|)")
print("  где |V| - размер словаря")
